In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Custom Libs
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../')))
from ridgepy.kalman_filter_network import KalmanFilterNetwork
from ridgepy.kalman_filter_network import KalmanFilterMode

In [3]:
# This example processes the single_mode_example file created by the
# bin/simulate/0.0-simulate-single-mode.py file.
data_df = pd.read_csv("single_mode_example.csv", sep=",", header=0)
data_df.head()

,time,phase,F,A,B,observation
0,0.000000,0.000000,150.0,1.0,0.5,1.000000
1,0.000063,0.058905,150.0,1.0,0.5,1.027701
2,0.000125,0.117810,150.0,1.0,0.5,1.051837
3,0.000188,0.176715,150.0,1.0,0.5,1.072325
4,0.000250,0.235619,150.0,1.0,0.5,1.089093


In [4]:
MODE_COUNT = 6
modes = []
for mode_number in range(1, MODE_COUNT + 1):
    # The signal noise covariance is a 2x2 matrix.
    signal_noise_covariance = np.zeros((2,2))
    signal_noise_covariance[0][0] = 0.0001
    signal_noise_covariance[1][1] = 0.0001

    signal_error_covariance = np.zeros((2,2))
    signal_error_covariance[0][0] = 1.0
    signal_error_covariance[1][1] = 1.0

    # The observation noise covariance is a scalar.
    observation_noise_covariance = 10.0

    # The coefficients are a 2 X 1 matrix.
    sin_coefficient = 0.0
    cos_coefficient = 0.0

    mode = KalmanFilterMode(
        mode_number,
        sin_coefficient,
        cos_coefficient,
        signal_error_covariance,
        signal_noise_covariance,
        observation_noise_covariance
    )
    modes.append(mode)

In [5]:
kf_network = KalmanFilterNetwork(modes)

2023-04-23 16:55:30,752 - INFO - None


KalmanFilterNetwork with 6 modes


In [6]:
time_step_data = []

for ndx in range(0, len(data_df)):
    ## Update the prior
    kf_network.prior_update(data_df.at[ndx, 'phase'])

    ## Update the posterior
    kf_network.posterior_update(data_df.at[ndx, 'observation'])

    ## Change the logging below to use a dictionary
    current_parameters = kf_network.current_parameters()

    time_step_data.append(current_parameters)

results_df = pd.DataFrame(time_step_data)

TypeError: KalmanFilterNetwork.prior_update() missing 1 required positional argument: 'phase'

In [ ]:
results_df.head()

In [ ]:
width  = 12.0
height = width / 1.618 / 2.

In [ ]:
fig, ax  = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=False, figsize=(width, height))
ax[0].plot(data_df['time'], data_df['observation'], linestyle='-', label=r'$Y(t)$')
ax[0].plot(data_df['time'], results_df['prediction'], linestyle='-', label=r'$\hat{h}(t)$')
ax[1].plot(data_df['time'], results_df['error'], linestyle='-', label=r'$e(t)$')
fig.legend(ncol=3, bbox_to_anchor=(0.5, 0.85))
ax[1].set_xlabel('time [seconds]')

In [ ]:
fig, ax  = plt.subplots(nrows=len(modes), ncols=4, sharex=True, sharey=True, figsize=(width, 3*height))
cos_coefficients = np.array(results_df['cos_coefficients'].apply(np.array).tolist())
sin_coefficients = np.array(results_df['sin_coefficients'].apply(np.array).tolist())
convergences = np.array(results_df['convergences'].apply(np.array).tolist())
magnitudes = np.array(results_df['magnitudes'].apply(np.array).tolist())
for mode_ndx in range(len(modes)):
    ax[mode_ndx,0].plot(data_df['time'], sin_coefficients[:,mode_ndx], linestyle='-', label=r'$\hat{A}^k(t)$')
    ax[mode_ndx,1].plot(data_df['time'], cos_coefficients[:,mode_ndx], linestyle='-', label=r'$\hat{B}^k(t)$')
    ax[mode_ndx,2].plot(data_df['time'], convergences[:,mode_ndx], linestyle='-', label=r'$\hat{r}^k(t)$')
    ax[mode_ndx,3].plot(data_df['time'], magnitudes[:,mode_ndx], linestyle='-', label=r'$\hat{P}^k(t)$')

In [ ]:
fs = 1./(data_df.at[1, 'time'] - data_df.at[0, 'time'])
plt.specgram(data_df['observation'], NFFT=256, Fs=fs, cmap='viridis')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram')
plt.colorbar(label='Power spectral density (dB/Hz)')
plt.show()

In [ ]:

fs = data_df.at[1, 'time'] - data_df.at[0, 'time']
magnitudes = np.array(results_df['magnitudes'].apply(np.array).tolist())
frequencies = np.array(results_df['frequencies'].apply(np.array).tolist())
magnitudes_db = np.log10(magnitudes) * 10.0
times = data_df['time']
print(times.shape, frequencies.shape, magnitudes_db.shape)
plt.pcolormesh(times, frequencies, magnitudes_db.T, cmap='viridis')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram')
plt.colorbar(label='Power spectral density (dB/Hz)')
plt.show()

In [ ]:
print(kf_network.frequencies.shape, freqs.shape)
print(magnitudes.shape)